In [1]:
import pandas as pd
import csv
import requests
import numpy as np
import json
import plotly.express as px
import plotly.io as pio


#Download_csv_file

CSV_URL = "https://api.covid19india.org/csv/latest/districts.csv"
with requests.Session() as s:
    download = s.get(CSV_URL)

    
#Read and write to csv

with open('covid19india_districts.csv', 'w') as f:
    writer = csv.writer(f)
    for line in download.iter_lines():
        writer.writerow(line.decode('utf-8').split(','))


#Creating Pivot table

districts=pd.read_csv('covid19india_districts.csv')
table = pd.pivot_table(districts, values='Confirmed', index=['District','State'], columns=['Date'], fill_value=0)
table.to_csv("district_pivot.tsv", sep='\t')
pivot_table=pd.read_csv("district_pivot.tsv", sep="\t")


#Creating two weeks data

two_weeks = pivot_table.T.tail(14).T
first_week = two_weeks.T.head(7).T
last_week = two_weeks.T.tail(7).T
first_column = pivot_table.T.head(2).T

first_week['avg_cases_previous_week'] = first_week.mean(axis=1)
last_week['avg_cases_over_7_days'] = last_week.mean(axis=1)
week1 = first_column.join(first_week)
week2 = first_column.join(last_week)

merged = week1.merge(week2, on=['District','State'])
merged['Increase'] = merged['avg_cases_over_7_days']-merged['avg_cases_previous_week']
merged['Rate_of_Increase']=(merged['Increase']/merged['avg_cases_previous_week'])*100


searchfor=['Airport Quarantine', 'BSF Camp', 'Capital Complex','Delhi', 'Evacuees', 'Gaurela Pendra Marwahi',
           'Italians', 'Other Region', 'Other State', 'Others', 'Railway Quarantine', 'Saitual', 'State Pool', 'Unknown', 
           'Mayiladuthurai', 'kra']
merged2 = merged[~merged.District.str.contains('|'.join(searchfor))]



header=['District','State','avg_cases_over_7_days','avg_cases_previous_week','Increase','Rate_of_Increase']
merged2.to_csv("district.tsv", sep='\t',columns=header)

#Creating image
pio.renderers.default = 'browser'

india_districts = json.load(open("india_districts.geojson", "r"))

district_id_map = {}
for feature in india_districts["features"]:
    feature["id"] = feature["properties"]["dt_code"]
    district_id_map[feature["properties"]["district"]] = feature["id"]


df = pd.read_csv("district.tsv",sep='\t')
df["id"] = df["District"].apply(lambda x: district_id_map[x])
df.loc[df['Rate_of_Increase'] <= 0, 'Increase_image'] = 0
df.loc[df['Rate_of_Increase'] > 0, 'Increase_image'] = df['Rate_of_Increase']



fig = px.choropleth(
    df,
    locations="id",
    geojson=india_districts,
    color="Increase_image",
    hover_name="District",
    hover_data=["Rate_of_Increase"],
    title="Rate of Increase in COVID19 cases per week / Date analyzed:31-10-2021",
    color_continuous_scale=px.colors.diverging.BrBG,
    color_continuous_midpoint=0)
fig.update_layout(coloraxis_colorbar=dict(
    title="Rate of Increase"),geo=dict(bgcolor= 'rgba(0,0,0,0)'))
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [3]:
df.sort_values(by=['Rate_of_Increase'], inplace=True, ascending=False, ignore_index=True)
df.loc[0:9, ['District','State', 'Rate_of_Increase']]

,District,State,Rate_of_Increase
0,Hnahthial,Mizoram,18.007939
1,Serchhip,Mizoram,12.723232
2,Champhai,Mizoram,6.994566
3,Lunglei,Mizoram,6.445651
4,Mamit,Mizoram,6.051734
5,Khawzawl,Mizoram,3.783102
6,Saiha,Mizoram,3.200414
7,Aizawl,Mizoram,2.963218
8,Idukki,Kerala,2.328204
9,Lawngtlai,Mizoram,2.176191
